# stock2vec

Create vectors for stocks based on their relative volatility.

In [194]:
import math
import numpy as np
import pandas as pd

### Explore

Let's see how the data looks.

In [199]:
def calculate_volatility(row):
    return np.sqrt((row.high - row.low) / row.close)

def pretty_print_ticker(row):
    volatility = calculate_volatility(row)
    print(row.ticker.values[0], '  ', row.date.values[0], ' ', volatility.values[0])

reader = pd.read_csv('data/wiki_prices.csv', chunksize=10000, iterator=True)

print('ticker     date     volatility')

for i in range(0,10):
    r = reader.get_chunk()
    pretty_print_ticker(r)

ticker     date     volatility
A    1999-11-18   0.476731294623
AAN    1991-01-10   0.0
AAON    2003-01-21   0.225701809375
AAPL    1991-03-13   0.237915475715
AAWW    2013-06-06   0.177802169323
ABC    2001-12-06   0.212854152101
ABCO    2003-07-18   0.169252636615
ABG    2006-04-24   0.142171607424
ABM    2013-05-02   0.157470995798
ABT    1989-06-05   0.111571200496
(14684262, 14)


## Preprocessing

The data is ordered by ticker but I want to find context across the same date.

So import the data into postgres with [pgfutter](https://github.com/lukasmartinelli/pgfutter).

```shell
DB_NAME=stocks pgfutter csv wiki_prices.csv
```

Convert the date column and add an index.

```sql
ALTER TABLE "import"."wiki_prices" ALTER COLUMN "date" 
  SET DATA TYPE date using(date::date);
  
CREATE INDEX idx_date ON import.wiki_prices(date);
```

Now we can build batches based on the date.

In [191]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://localhost:5432/stocks')

def get_stocks_by_year(year):
    dateCondition = "date >= '" + str(year) + "-01-01'::date AND date < '" + str(year + 1) + "-01-01'::date"
    
    query = """SELECT ticker, date, 
    abs((high::numeric(12,4) - low::numeric(12,4)) / close::numeric(12,4)) as volt
    FROM import.wiki_prices WHERE """ + dateCondition + """
    AND high != '' AND low != '' AND close != '' AND close::numeric(12, 4) > 0
    ORDER BY date, volt DESC"""
    
    return pd.read_sql_query(query, con=engine)

stocks_year = get_stocks_by_year(2010)

print(stocks_year.head(), stocks_year.shape)

  ticker        date      volt
0   NEON  2010-01-04  0.500000
1   TXMD  2010-01-04  0.326667
2   INSY  2010-01-04  0.312500
3   YRCW  2010-01-04  0.242857
4   MILL  2010-01-04  0.224000 (667694, 3)


**Build Batches**

For each stock, find C stocks that have the closest volatility to that ticker for that day.

In [193]:
ticker_to_int = {}
int_to_ticker = {}

def get_ticker_int(ticker):
    key = ticker_to_int.get(ticker, None)
    if key is None:
        key = ticker_to_int[ticker] = len(ticker_to_int)
        int_to_ticker[key] = ticker
    return key

def get_window(stocks, idx, window_size=5):
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    
    stock = stocks.values[idx]
    ticker_int = get_ticker_int(stock[0])
    date = stock[1]
    
    window = []
    
    for i in range(start, stop+1):
        context_stock = stocks.values[i]
        context_ticker_int = get_ticker_int(context_stock[0])
        if context_ticker_int != ticker_int and context_stock[1] == date:
            window.append(context_ticker_int)
    
    return window

for idx in range(0, 5):
    print(train_stocks.values[idx][0])
    for i in get_window(train_stocks, idx, 5):
        print(i, ' ', int_to_ticker[i])

NEON
1   TXMD
2   INSY
3   YRCW
TXMD
0   NEON
2   INSY
3   YRCW
4   MILL
5   ESCA
INSY
0   NEON
1   TXMD
3   YRCW
4   MILL
YRCW
0   NEON
1   TXMD
2   INSY
4   MILL
5   ESCA
6   BPZ
7   WSBF
8   IVAC
MILL
3   YRCW
5   ESCA


## Vector Math

Apple - Google = ?